In [229]:
# imports
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    LOC,
    ORG,
    NamesExtractor,
    DatesExtractor,
    AddrExtractor,
    Doc
)
from natasha.grammars.date import YEAR

from yargy import Parser, rule, or_, and_
from yargy.predicates import gram, dictionary, gte, lte, type, eq, is_capitalized, true, custom
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact

In [72]:
with open('patrologia_latina.txt') as file:
    text = file.read()
text

'Patrologia Latina (Латинская патрология), или Patrologiae Cursus Completus — собрание сочинений латиноязычных христианских авторов, включающее 217 огромных томов, первая часть «Полного курса патрологии» (Patrologiæ Cursus Completus), вторая часть — Patrologia Græca. Изданы аббатом Минем в 1844—1855 годах, в 1862—1865 опубликованы указатели. Охватывает период примерно в тысячу лет — с конца II по начало XIII века. Первоначально Минь планировал опубликовать все сочинения вплоть до эпохи Возрождения, но объём издания оказался слишком велик. Издание отличает полнота жанров: включены богословские труды, агиография, поэзия, историческая и научная литература, переводы с греческого, переписка, материалы соборов, различная документация. Корпус представляет собой по сути огромную перепечатку ранее опубликованных изданий и не имеет самостоятельного текстологического значения[1]. Значительное число произведений с тех пор не переиздавалось, и издание продолжает использоваться медиевистами, хотя в 

In [73]:
# text filtering
text = text.replace('\t', ' ')
text

'Patrologia Latina (Латинская патрология), или Patrologiae Cursus Completus — собрание сочинений латиноязычных христианских авторов, включающее 217 огромных томов, первая часть «Полного курса патрологии» (Patrologiæ Cursus Completus), вторая часть — Patrologia Græca. Изданы аббатом Минем в 1844—1855 годах, в 1862—1865 опубликованы указатели. Охватывает период примерно в тысячу лет — с конца II по начало XIII века. Первоначально Минь планировал опубликовать все сочинения вплоть до эпохи Возрождения, но объём издания оказался слишком велик. Издание отличает полнота жанров: включены богословские труды, агиография, поэзия, историческая и научная литература, переводы с греческого, переписка, материалы соборов, различная документация. Корпус представляет собой по сути огромную перепечатку ранее опубликованных изданий и не имеет самостоятельного текстологического значения[1]. Значительное число произведений с тех пор не переиздавалось, и издание продолжает использоваться медиевистами, хотя в 

In [85]:
#определяем необходимые функции для работы с текстом
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb) # морфологический разбор слова
syntax_parser = NewsSyntaxParser(emb) # анализ синтаксиса 
ner_tagger = NewsNERTagger(emb) #извлечение именованных сущностей

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [86]:
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

In [213]:
persons = []
dates = []
organisations = []
for span in doc.spans:
    span.normalize(morph_vocab)
    if span.type == PER:
        span.extract_fact(names_extractor)
        if span.fact:
            persons.append(span.fact.as_dict)
    if span.type == ORG:
        organisations.append(span.normal)

for date_match in dates_extractor(text):
    dates.append(dates.append(date_match.fact))

In [96]:
# Persons
persons

[{'first': 'Минь'},
 {'first': 'Минь'},
 {'first': 'Тертуллиан'},
 {'first': 'Феликс', 'last': 'Минуций'},
 {'first': 'Киприану'},
 {'first': 'Максим'},
 {'first': 'Сидоний'},
 {'first': 'Махарий'},
 {'first': 'Корнелий'},
 {'first': 'Луций', 'last': 'I'},
 {'first': 'Стефан', 'last': 'I'},
 {'first': 'Киприан', 'last': 'Карфагенский'},
 {'first': 'Дионисий', 'last': 'Александрийский'},
 {'first': 'Феликс', 'last': 'I'},
 {'first': 'Евтихиан'},
 {'first': 'Гай'},
 {'first': 'Коммодиан'},
 {'first': 'Викторин'},
 {'first': 'Магнет'},
 {'first': 'Марцеллин'},
 {'first': 'Евсевий'},
 {'first': 'Мильтиад'},
 {'first': 'Константин', 'last': 'I'},
 {'first': 'Евмений'},
 {'first': 'Сильвестр', 'last': 'I'},
 {'first': 'Марк'},
 {'first': 'Юлий', 'last': 'I'},
 {'first': 'Викторин', 'last': 'Африканец'},
 {'first': 'Осий', 'last': 'Кордовский'},
 {'first': 'Либерий'},
 {'first': 'Потамий'},
 {'first': 'Иларий'},
 {'first': 'Зенон', 'last': 'Веронский'},
 {'last': 'Милевитанский'},
 {'first': 

In [214]:
# Dates
dates

[Date(
     year=1855,
     month=None,
     day=None
 ),
 None]

In [99]:
# Organisations
organisations

['Patrologia Latina (Латинская патрология',
 'Карфагенский собор',
 'Халкидонский собор',
 'Парадиз',
 'Хроника',
 'Безансона',
 'Франкфуртский синод',
 'Генеалогия Карл Великий',
 'Лондонский собор',
 'Кведлинбургские анналы',
 'Chronicon Petrishuensis',
 'Деяния святых мучеников Арнальда и Герлембальда',
 'Майнцский собор',
 'Панегирик император Беренгарию',
 'Деяния епископов Меца',
 'Анналы св. Витона Верденского',
 'Одо',
 'Во-де-Серне']

In [171]:
person_fact = fact(
    'Person',
    ['name', 'position']
)

position_fact = fact(
    'Position',
    ['position']
)

name_fact = fact(
    'Name',
    ['first', 'last', 'number', 'nickname']
)

post_rule = rule(
    '(', 
    gram('NOUN').repeatable().interpretation(position_fact.position), 
    ')'
).interpretation(position_fact)

name_rule = rule(
    and_(gram('Name'), is_capitalized()).interpretation(name_fact.first),
    and_(gram('NOUN'), is_capitalized()).repeatable().optional().interpretation(name_fact.last),
    type('LATIN').optional().interpretation(name_fact.number),
    gram('ADJF').optional().interpretation(name_fact.nickname),
).interpretation(name_fact)

person_rule = rule(
    name_rule.interpretation(person_fact.name), 
    post_rule.optional().interpretation(person_fact.position),
).interpretation(person_fact)

person_parser = Parser(person_rule)

In [173]:
# Better persons
for sent in doc.sents:
    parsed = person_parser.find(sent.text)
    if parsed:
        print(parsed.fact)

Person(name=Name(first='Минем', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Минь', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='В', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Тертуллиан', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Феликс', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Адама', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Корнелий', last=None, number=None, nickname=None), position=Position(position='папа римский'))
Person(name=Name(first='Луций', last=None, number='I', nickname=None), position=Position(position='папа римский'))
Person(name=Name(first='Стефан', last=None, number='I', nickname=None), position=Position(position='папа римский'))
Person(name=Name(first='Морцелл', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Дионисий', las

Person(name=Name(first='Адревальд', last=None, number=None, nickname=None), position=Position(position='монах Флёри'))
Person(name=Name(first='Лана', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Исаак', last=None, number=None, nickname=None), position=Position(position='епископ Лангрский'))
Person(name=Name(first='Бовэ', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Хейрик', last=None, number=None, nickname='Осерский'), position=None)
Person(name=Name(first='Иоанн', last=None, number='VIII', nickname=None), position=Position(position='папа римский'))
Person(name=Name(first='Марин', last=None, number='I', nickname=None), position=Position(position='папа римский'))
Person(name=Name(first='Адриан', last=None, number='III', nickname=None), position=Position(position='папа римский'))
Person(name=Name(first='Бертарий', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Хармот', last=None, number=No

Person(name=Name(first='Гислеберт', last='Криспин', number=None, nickname=None), position=Position(position='аббат Вестминстера'))
Person(name=Name(first='Беренгоз', last=None, number=None, nickname=None), position=Position(position='аббат Трирский'))
Person(name=Name(first='Иоанн', last='Марсикан', number=None, nickname=None), position=Position(position='епископ Тускульский'))
Person(name=Name(first='Родульф', last='Тортарий', number=None, nickname=None), position=None)
Person(name=Name(first='Франциск', last='Камен', number=None, nickname=None), position=None)
Person(name=Name(first='Анселл', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Реймбальд', last=None, number=None, nickname=None), position=None)
Person(name=Name(first='Иоанн', last=None, number=None, nickname=None), position=Position(position='монах'))
Person(name=Name(first='Пётр', last='Хрисолан', number=None, nickname=None), position=None)
Person(name=Name(first='Роберт', last=None, number=

In [218]:
titled_books_fact = fact(
    'Book',
    ['title']
)

titled_books_rule = rule(
    '«',
    custom(lambda s: s != '»').repeatable().interpretation(titled_books_fact.title),
    '»'
).interpretation(titled_books_fact)

titled_books_parser = Parser(titled_books_rule)

In [192]:
# Books
for sent in doc.sents:
    parsed = titled_books_parser.find(sent.text)
    if parsed:
        print(parsed.fact)

Book(title='Полного курса патрологии')
Book(title='Мученичество святых Перпетуи и Фелицитаты')
Book(title='Книга происхождения от Адама')
Book(title='Трактат к еретикам новацианам')
Book(title='Песнь против язычников')
Book(title='Песнь во славу Господа')
Book(title='Хроника')
Book(title='Шестоднева')
Book(title='Хронологический указатель о преследованиях вандалов')
Book(title='О франкских законах')
Book(title='Предисловие к житию св. Павла')
Book(title='Житие св. Онуфрия')
Book(title='Житие св. Василия Кесарийского')
Book(title='Житие св. Симеона Стилита')
Book(title='Житие св. Иоанна Милостивого')
Book(title='Житие св. Макария Римского')
Book(title='Житие св. Варлаама и Иоасафа')
Book(title='Житие св. Марии Блудницы')
Book(title='Житие св. Пелагии Блудницы')
Book(title='Житие св. Марии Египетской блудницы')
Book(title='Verba Seniorum')
Book(title='Паломничество')
Book(title='Духовный луг')
Book(title='Парадиз')
Book(title='Лавсаик')
Book(title='Хроника')
Book(title='Феофил')
Book(tit

In [232]:
period_fact = fact(
    'Period',
    ['date_from', 'date_to']
)

period_rule = rule(
    YEAR.interpretation(period_fact.date_from),
    '—',
    YEAR.interpretation(period_fact.date_to)
).interpretation(period_fact)

period_parser = Parser(period_rule)

In [233]:
# Date periods
for sent in doc.sents:
    parsed = period_parser.find(sent.text)
    if parsed:
        print(parsed.fact)

Period(date_from=1844, date_to=1855)
Period(date_from=1220, date_to=1246)
Period(date_from=1091, date_to=1179)
Period(date_from=1034, date_to=1467)
Period(date_from=1147, date_to=1192)
